In [5]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 150},
)

chain = prompt | llm

chain.invoke({"word": "tomato"})

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


" soft, sweet, moist, succulent and crunchy tomato that is best served when served raw. It is also easy to prepare by mixing together half and half.\n\nThe first ingredient is ground nuts. This can be found in most grocery stores in the U.S but also is common in a wide range of vegetables. This is a very important ingredient to have in your kitchen. These nuts contain sodium but this is not essential.\n\nThe second ingredient is sugar. If you want to avoid the dreaded syrup, you can buy sugar from Whole Foods in small amounts or from Trader Joe's. These are generally inexpensive and you can expect to consume them daily. Sugar is rich in antioxidants - it may also have a protective effect against parasites and"